<a href="https://colab.research.google.com/github/GaCampioto/tech-challenges-fiap-AI-for-developers/blob/main/tech_challenge_3_fine_tuning_steps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine tuning steps

Devido ao dataset disponibilizado ser muito grande, tive que quebrá-lo em chunks para que o treinamento fosse realizado em chunks e não com o arquivo completo.

Este notebook foi executado em uma A100 RAM alta

In [ ]:
# habilitando a utilização dos arquivos do drive como fonte de dados
from google.colab import drive
drive.mount('/content/drive')

tc3_path = '/content/drive/MyDrive/Colab Notebooks/tech-challenge-3'

Mounted at /content/drive


## Instalando dependências, definindo configurações e parâmetros

In [ ]:
# alterando a versão do torch do colab devido a problemas com unsloth e xformers
!pip uninstall torch -y
!pip install torch==2.4.0

import torch
torch.__version__

Found existing installation: torch 2.4.0+cu121
Uninstalling torch-2.4.0+cu121:
  Successfully uninstalled torch-2.4.0+cu121
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 95.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 13.0 MB/s eta 0:0

'2.4.0+cu121'

In [ ]:
# instalação do Unsloth, Xformers e outros pacotes
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes tritonserver
!pip install transformers datasets
!pip install xformers
!pip install trl
!pip install peft
!pip install accelerate
!pip install bitsandbytes
!pip install tritonserver

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-hnw1rrw9/unsloth_48ad402fc0d441a69c87fc28d3cc3cac
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-hnw1rrw9/unsloth_48ad402fc0d441a69c87fc28d3cc3cac
  Resolved https://github.com/unslothai/unsloth.git to commit d91d40a7b6b556f2d1fdd3e1e430f7a76a799627
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 23.8 MB/s eta 0:00:00
   

In [ ]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
import json
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

max_seq_length = 2048
dtype = None
load_in_4bit = True

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


## Importando o modelo

In [ ]:
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = f"{tc3_path}/lora_model", # como fizemos o treinamento do 1o chunk no notebook anterior, podemos recuperar um modelo pré existente
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4bit(
      

Da mesma maneira que no notebook anterior, vamos formatar os dados de cada chunk, para que seja possível executar o treinamento.

In [ ]:
prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):

        text = prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset

train_dss = []
for i in range(1, 11):
  train_ds = load_dataset("json", data_files=f"{tc3_path}/formatted-data-{i}.json", split='train')
  train_ds = train_ds.map(formatting_prompts_func, batched = True)
  train_dss.append(train_ds)

print(train_dss)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/139041 [00:00<?, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/139041 [00:00<?, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

...

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/139040 [00:00<?, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/139040 [00:00<?, ? examples/s]

[Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 139041
}), Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 139041
}), Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 139041
}), Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 139040
}), Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 139040
}), Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 139040
}), Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 139040
}), Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 139040
}), Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 139040
}), Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 139040
})]


Neste ponto vamos criar 1 `trainer` para cada chunk

In [ ]:
trainers = []

for train_ds in train_dss:
  trainer = SFTTrainer(
      model = model,
      tokenizer = tokenizer,
      train_dataset = train_ds,
      dataset_text_field = "text",
      max_seq_length = max_seq_length,
      dataset_num_proc = 2,
      packing = False,
      args = TrainingArguments(
          per_device_train_batch_size = 2,
          gradient_accumulation_steps = 4,
          warmup_steps = 5,
          max_steps = 60,
          learning_rate = 2e-4,
          fp16 = not is_bfloat16_supported(),
          bf16 = is_bfloat16_supported(),
          logging_steps = 1,
          optim = "adamw_8bit",
          weight_decay = 0.01,
          lr_scheduler_type = "linear",
          seed = 3407,
          output_dir = "outputs",
      ),
  )

  trainers.append(trainer)


Map (num_proc=2):   0%|          | 0/139041 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


Map (num_proc=2):   0%|          | 0/139041 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


Map (num_proc=2):   0%|          | 0/139041 [00:00<?, ? examples/s]

...

max_steps is given, it will override any value given in num_train_epochs


Map (num_proc=2):   0%|          | 0/139040 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


Map (num_proc=2):   0%|          | 0/139040 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


Por fim vamos treinar cada `trainer` e no final salvar o modelo no Google Drive para que seja possível importá-lo no notebook anterior e avaliar a diferença entre os resultado pré treinamento e pós treinamento.

In [ ]:
for trainer in trainers:
  trainer.train()

model.save_pretrained(f"{tc3_path}/lora_model")
tokenizer.save_pretrained(f"{tc3_path}/lora_model")

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 139,041 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.870000
2,1.834000
3,1.638400
4,1.733500
5,2.143800
6,1.950000
7,1.751800
8,2.165200
9,1.958900
10,1.846100


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 139,041 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.706600
2,1.677400
3,2.146800
4,1.717200
5,1.681100
6,1.980800
7,1.780200
8,1.957200
9,1.695700
10,1.720100


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 139,041 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


...

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 139,040 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.730400
2,1.756500
3,1.670400
4,1.604500
5,1.902500
6,1.922900
7,1.965200
8,1.823200
9,1.787700
10,1.954700


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 139,040 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.755900
2,1.855700
3,1.803900
4,1.888800
5,1.786600
6,1.774600
7,1.638000
8,1.706500
9,1.690100
10,1.759700


('/content/drive/MyDrive/Colab Notebooks/tech-challenge-3/lora_model/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/tech-challenge-3/lora_model/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/tech-challenge-3/lora_model/tokenizer.json')